In [82]:
import pandas as pd

## Objectif du projet : 
#### Prédire le nombre de validations par jour pour toutes les gares de SNCF-Transilien entre le 1er janvier 2023 et le 30 juin 2023. 


#### x-train est un jeu de données avec 1 237 971 lignes et 6 colonnes. Il s’agit des données quotidiennes de validations des 448 stations du réseau SNCF-Transilien (comprenant toutes les gares du RER A et du RER B) entre le 1er janvier 2015 et le 31 décembre 2022 (soit 2922 jours).

#### et y-train c'est le nombre de validations par jour pour toutes les gares de SNCF-Transilien entre le 1er janvier 2015 et le 31 décembre 2022 (soit 2922 jours).

In [83]:
x_train = pd.read_csv("Data/train_f_x.csv")
y_train =  pd.read_csv("Data/y_train_sncf.csv")
x_test = pd.read_csv("Data/test_f_x_THurtzP.csv")
x_train.head()

,date,station,job,ferie,vacances
0,2015-01-01,1J7,1,1,1
1,2015-01-01,O2O,1,1,1
2,2015-01-01,8QR,1,1,1
3,2015-01-01,UMC,1,1,1
4,2015-01-01,FK3,1,1,1


In [84]:
## x_train['date'] = pd.to_datetime(x_train['date'])
## x_train = x_train.set_index("date")
## x_train.loc["2022-01-31"] 

## Phase 1 : Analyse de données et prétraitement 